In [37]:
import jax
import jax.numpy as np
import numpy as onp
from lbi.pipeline.base import pipeline
from lbi.prior import SmoothedBoxPrior
import h5py
import limits_plots as lp
import corner
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
plot_dir = "../../plots/"

# Load in array
logger = None
hf = h5py.File('samples_and_results.h5', 'r')
samples = hf['samples'][:]
results = {}
for key in hf.keys():
    if key != 'samples':
        results[key] = hf[key][:]
hf.close()

In [3]:
results['mneut1'] = onp.abs(results['mneut1'])

# Plot arguments 
lower_limits = onp.array([[0.01, 74e-11, 122.0, 2 * 0.02275]])
upper_limits = onp.array([[0.30, 428e-11, 128.0, 1.1]])
limits = onp.stack([lower_limits, upper_limits]).squeeze()
ranges = [
    [-2000, 2000],
    [-1000, 1000],
    [-2000, 2000],
    [400, 4000],
    [100, 4000],
    [100, 1000],
    [100, 4000],
    [100, 4000],
    [100, 1000],
    [100, 4000],
    [400, 4000],
    [1, 60],
    [-4000, 4000],
    [-4000, 4000],
    [-4000, 4000],
]
labels = [
    r"$\mu$",
    r"$M_1$",
    r"$M_2$",
    r"$M_3$",
    r"$M_{L_1}$",
    r"$M_{L_2}$",
    r"$M_{L_3}$",
    r"$M_{r_1}$",
    r"$M_{r_2}$",
    r"$M_{r_3}$",
    # r"$M_{Q_1}$",
    # r"$M_{Q_2}$",
    # r"$M_{Q_3}$",
    # r"$M_{u_1}$",
    # r"$M_{u_2}$",
    # r"$M_{u_3}$",
    # r"$M_{d_1}$",
    # r"$M_{d_2}$",
    # r"$M_{d_3}$",
    r"$M_A$",
    r"$\tan\beta$",
    r"$A_t$",
    r"$A_b$",
    r"$A_{\tau}$",
]
levels = [0.6827, 0.9545, 0.9973]

X_true = onp.array([[0.12, 251e-11, 125.0, 0.05]])
X_sigma = onp.array([0.01, 59e-11, 1.0, 1e-5])

In [4]:
filtered_unitful_samples, filtered_results = lp.filter_observables(
    samples, results, limits
)

In [28]:
lp.plot_direct_detection_limits(results, filename="unfiltered_direct_detection.png")
lp.plot_direct_detection_limits(filtered_results, filename="direct_detection.png")

In [26]:
lp.plot_mass_splitting(results, filename="unfiltered_LHC_mass_splitting_constraints.png")
lp.plot_mass_splitting(filtered_results)


In [60]:
plt.subplots(figsize=(10, 8))
for i,label in enumerate(labels):
    plt.subplot(3, 5, i+1)
    plt.hist(samples[:,i])
    # plt.hist(filtered_unitful_samples[:,i])
    if "tan" not in label:
        plt.xlabel(label + " (GeV)")
    else:
        plt.xlabel(label)
    plt.xlim(ranges[i])
    plt.gca().get_yaxis().set_visible(False)
    plt.box(False)
plt.tight_layout()

/var/folders/qd/09mj47cx551c65glw6y47d6r0000gn/T/ipykernel_53116/1127304935.py:3: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(3, 5, i+1)
